In [ ]:
class Solution:

    def countOperationsToEmptyArray(self, nums: List[int]) -> int:
        # pre <= i  query(pre, i)-1    query(i, j) [i, j]有效元素个数
        # i < pre  query(pre, n - 1) + query(0, i) - 1
        pre = 0
        n = len(nums)
        ans = n
        val = [0] * (4 * n)

        def build(o, l, r):
            if l == r:
                val[o] = 1
                return
            m = l + r >> 1
            build(o * 2, l, m)
            build(o * 2 + 1, m + 1, r)
            val[o] = val[o * 2] + val[o * 2 + 1]

        def update(o, l, r, idx, v):  # 单点设置
            if l == r:
                val[o] = v
                return
            m = l + r >> 1
            if m >= idx:
                update(o * 2, l, m, idx, v)
            else:
                update(o * 2 + 1, m + 1, r, idx, v)
            val[o] = val[o * 2] + val[o * 2 + 1]

        def query(o, l, r, L, R):  # [L,R]区间查询
            if r < L or l > R:
                return 0
            if L <= l and r <= R:
                return val[o]
            m = l + r >> 1
            return query(o * 2, l, m, L, R) + query(o * 2 + 1, m + 1, r, L, R)

        build(1, 0, n - 1)

        for i, _ in sorted(enumerate(nums), key=lambda x: x[1]):
            if pre <= i:
                ans += query(1, 0, n - 1, pre, i) - 1
            else:
                ans += query(1, 0, n - 1, pre, n - 1) + query(1, 0, n - 1, 0, i) - 1
            update(1, 0, n - 1, i, 0)
            pre = i

        return ans

In [ ]:
# 超时
class Node:

    def __init__(self, si, ei):
        self.si, self.ei, self.mi = si, ei, si + ei >> 1
        self.lgh = ei - si + 1
        self.left = self.right = None
        self.lazy = None  # 当前节点设为None未下发
        self.val = 0  # 区间和


class SegmentTree:

    def __init__(self, si, ei):
        self.root = Node(si, ei)

    def push_down(self, node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if node.lazy is None: return
        node.left.val += node.left.lgh * node.lazy
        node.right.val += node.right.lgh * node.lazy
        node.left.lazy = node.right.lazy = node.lazy
        node.lazy = None

    def update(self, idx, val):
        self._update(self.root, idx, val)

    def _update(self, node, idx, val):  # 单点设置
        if node.si == node.ei:
            node.val = val
            return
        self.push_down(node)
        if node.mi >= idx:
            self._update(node.left, idx, val)
        else:
            self._update(node.right, idx, val)
        node.val = node.left.val + node.right.val

    def query(self, l, r):
        return self._query(self.root, l, r)

    def _query(self, node, l, r):  # 查询[l,r]区间和
        if r < node.si or l > node.ei or l > r:
            return 0
        if l <= node.si and node.ei <= r:
            return node.val
        self.push_down(node)
        return self._query(node.left, l, r) + self._query(node.right, l, r)


class Solution:

    def countOperationsToEmptyArray(self, nums: List[int]) -> int:
        # pre <= i  query(pre, i)-1    query(i, j) [i, j]有效元素个数
        # i < pre  query(pre, n - 1) + query(0, i) - 1
        pre = 0
        n = len(nums)
        ans = n
        st = SegmentTree(0, n - 1)
        for i in range(n):
            st.update(i, 1)
        for i, _ in sorted(enumerate(nums), key=lambda x: x[1]):
            if pre <= i:
                ans += st.query(pre, i) - 1
            else:
                ans += st.query(pre, n - 1) + st.query(0, i) - 1
            # print(ans,pre,i,st.query(pre, i) - 1)
            st.update(i, 0)
            pre = i
        return ans

In [4]:
from sortedcontainers import SortedList

sl = SortedList([1, 1, 2, 3])
sl.discard(1), sl.bisect_left()


(None, SortedList([1, 2, 3]))